In [2]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import re
from nltk.corpus import stopwords
import nltk
import string
import matplotlib.pyplot as plt
import random
%matplotlib inline
import os
import json

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import decomposition, ensemble
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
#from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

import joblib

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import os,boto3,sys,glob,json

### Part 1  Loading the  data 

#### Read in csv file

In [3]:
testDF=pd.read_csv('run1000charts_0521.csv')
#testDF= pd.read_csv('/home/jovyan/work/Analytics_Data_training/bfAI_allinst_7502charts_061019.csv') # 7502 charts

# the csv is generated in D drive D:/chartai_qa  with extract1000charts_jsontxt.py, then uploaded to here

In [4]:
len(testDF)

76792

In [5]:
testDF.columns

Index(['Unnamed: 0', 'chart_id', 'code', 'text', 'start_offset', 'end_offset',
       'text75'],
      dtype='object')

In [6]:
testDF=testDF.drop(['Unnamed: 0'],axis=1)

In [7]:
testDF.head()

,chart_id,code,text,start_offset,end_offset,text75
0,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E063,Ht,1187,1189,ocedure codes 36415 venipunct routine vital si...
1,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E785,Hyperlipidemia,1374,1388,tion list reviewed reconciled patient past med...
2,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,I10,Hypertension,1391,1403,reconciled patient past medical history hyperl...
3,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E785,Hyperlipidemia,2957,2971,tion list reviewed reconciled patient past med...
4,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,I10,Hypertension,2974,2986,reconciled patient past medical history hyperl...


In [8]:
# just check if there's null text, if yes, remove them 
nulltext=testDF[~pd.notnull(testDF['text'])] 
len(nulltext)

0

In [9]:
nulltext=testDF[~pd.notnull(testDF['text75'])] 
len(nulltext)

26

In [10]:
nulltext   # there are some text not correctly extracted

,chart_id,code,text,start_offset,end_offset,text75
1336,IBXRMA2018_MULT_1164353980010001_HMK_160303709...,G129,SMA,4,7,NaN
2198,IBXRMA2018_MULT_1176161080010001_HMK_192365603...,N08,Kidney,46,52,NaN
3876,IBXRMA2018_MULT_1202740400010001_HMK_153425750...,G129,SMA,4,7,NaN
4032,IBXRMA2018_MULT_1202908350010001_HMK_210648786...,E744,PC,25,27,NaN
5591,IBXRMA2018_MULT_1210779380010001_HMK_184462858...,G129,SMA,4,7,NaN
6465,IBXRMA2018_MULT_1212745310010001_HMK_165441851...,N08,Kidney,48,54,NaN
9116,IBXRMA2018_MULT_1216729130010001_HMK_196286386...,N08,Kidney,46,52,NaN
14970,IBXRMA2018_MULT_1216856710010001_HMK_173309591...,G129,SMA,4,7,NaN
18950,IBXRMA2018_MULT_1216874020010001_HMK_143364782...,E744,PC,25,27,NaN
21748,IBXRMA2018_MULT_1216901800010001_HMK_201224196...,G129,SMA,4,7,NaN


In [11]:
testDF=testDF[pd.notnull(testDF['text75'])] 
len(testDF)

76766

In [12]:
testDF['chart_id'].nunique()   # only 980 charts not 1044 charts

980

In [13]:
testDF=testDF.reset_index(drop=True)   # very very import, to reset the index, otherwise the below running the model will not be right, because I use the index as reference for position and matching them
testDF.tail()

,chart_id,code,text,start_offset,end_offset,text75
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...


### Part 2 Loading all the models or locate the directory

#### Part 2.a. Load the 28 pickle files and run them on the testDF with selected chart_ids

In [508]:
#files=glob.glob("../../picklefolder_ngrams/refreshed_models_update0531/*.pickle") # refreshed updated models
files=glob.glob("../../picklefolder_ngrams/refreshed_models/*19.pickle")  # refreshed models
#files=glob.glob("../../picklefolder_ngrams/pipeline_pickles/*.pickle")  # pipeline models

In [509]:
files

['../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_hypertension_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_dementia_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cholesterol_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cad_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_aneurysm_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_COPD_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Depression_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Diabetes_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Heart_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Kidney_LRApril0219.pickle',
 '../../picklefolder_ngrams/refr

In [510]:
#files=files[:-1]
len(files)

28

In [23]:
files[27].split('/')[-1].split('_')[1]

'Cancer'

In [24]:
files[10].split('/')[-1]

'Phrase_Depression_Mar5.pickle'

In [25]:
files[24]

'../../picklefolder_ngrams/pipeline_pickles/Code_Diabetes_Mar5.pickle'

### Part 3. Apply models to test data

In [26]:
#encoder = preprocessing.LabelEncoder()
#test_y = encoder.fit_transform(testDF['flag'])  


In [511]:
code_list={}
code_list["Model1"]=['I10','E119']
code_list["Model2"]=['I2510','E785','K219','F329','I639']
code_list["Model3"]=['I739','N183','Z992','B20','I213','R569','I43','J410','I714','R579','E550','I209','J45998','I480','B182','K210','K739']
code_list["Heart"]=['I509','I482','I4891','I82401']
code_list["Depression"]=['F411','F410','F40240','F458','F409','F4000','F4001','F445','F444','F40243','F446','F4010','F442','F408','F449','F451']
code_list["Kidney"]=['N189','N181','N182','E1122','I130','E0822','E1022']
code_list["Diabetes"]=['Z794','E109','E119','E139','E089','E099']
code_list["Cancer"]=['C801','C50919','C189']
code_list["COPD"]=['J449','J45909']
code_list["Cholesterol"]=['E780','E785','E782','E789','E7800','E784','E781','E7801','E7881','E882','E783','E786','E7889','E756','E755','E7130']

In [512]:
allpredict=pd.DataFrame()   # create an empty dataframe, to save the prediction result for each model
allprob=pd.DataFrame()
print(allprob)

Empty DataFrame
Columns: []
Index: []


In [29]:
# for one model evaluation
'''
indexdf=pd.DataFrame() 
model_pipeline = joblib.load(files)
  #  f = open("allmodeltestscore_0506.txt", "a+")
    # deletion model

testDF1=testDF
indexdf[0]= [1 for i in range(len(testDF))]
        
        
        #print(files[i])
    # how to put the prediction on the right index?
test=testDF1['text75']
print("model loaded:", files )
print("length of data for the model:", len(test))
    
   
proba = model_pipeline.predict_proba(test)[:,1]
prediction = np.where(proba > 0.3, 1, 0) # use a lower threshold
testDF1['key']=testDF1.index
testDF1['pred']=prediction
testDF1['prob']= proba

indexdf['key']=indexdf.index
merged=pd.merge(indexdf,testDF1,on='key',how='outer')  # maintain the position of the perdiction, match to the right index
allpredict[0]=merged['pred']    # save the prediction to corresponding dataframe
allprob[0]=merged['prob']
'''

'\nindexdf=pd.DataFrame() \nmodel_pipeline = joblib.load(files)\n  #  f = open("allmodeltestscore_0506.txt", "a+")\n    # deletion model\n\ntestDF1=testDF\nindexdf[0]= [1 for i in range(len(testDF))]\n        \n        \n        #print(files[i])\n    # how to put the prediction on the right index?\ntest=testDF1[\'text75\']\nprint("model loaded:", files )\nprint("length of data for the model:", len(test))\n    \n   \nproba = model_pipeline.predict_proba(test)[:,1]\nprediction = np.where(proba > 0.3, 1, 0) # use a lower threshold\ntestDF1[\'key\']=testDF1.index\ntestDF1[\'pred\']=prediction\ntestDF1[\'prob\']= proba\n\nindexdf[\'key\']=indexdf.index\nmerged=pd.merge(indexdf,testDF1,on=\'key\',how=\'outer\')  # maintain the position of the perdiction, match to the right index\nallpredict[0]=merged[\'pred\']    # save the prediction to corresponding dataframe\nallprob[0]=merged[\'prob\']\n'

In [514]:
#f = open("allmodeltestscore_0506.txt", "w+")  # save all the score to this file
# for multiple model evaluation

   # create a dataframe, otherwise can use testDF for merging to obtain the prediction matched to index
# because for some model testing, we only select partial data, like phrase-based model, code-based model
for i in range(len(files)):
    indexdf=pd.DataFrame() 
    model_pipeline = joblib.load(files[i])
  #  f = open("allmodeltestscore_0506.txt", "a+")
    # deletion model
    if(files[i].split('/')[-1].startswith('D')):
        try:
            testDF1=testDF
            indexdf[i]= [1 for i in range(len(testDF))]
        except:
            print('no data for this model')
        
    elif(files[i].split('/')[-1].startswith('P')):
    # if it's phrase-based model, use the following way to select data
        
        phrase=files[i].split('/')[4].split('_')[1]
        try:
            testDF1 = testDF[testDF['text'].str.contains(phrase,flags=re.IGNORECASE)]
            temp=testDF1.index.tolist()
            indexdf[i]=[1 if i in (temp) else 0 for i in range(len(testDF))]
        except:
            print('no data for this model')

    elif(files[i].split('/')[-1].startswith('C')):
    # if it's code based model, use the following way to select model
        codereason=files[i].split('/')[4].split('_')[1]
        try:
            testDF1 = testDF[testDF['code'].isin(code_list[codereason])]
            temp=testDF1.index.tolist()
            indexdf[i]=[1 if i in (temp) else 0 for i in range(len(testDF))]
        except:
            print('no data for this model')
    else:
        print('not a valid/good model name probably...')

    print(files[i])
    # how to put the prediction on the right index?
    test=testDF1['text75']
    print("length of data for the model:", len(test))
    try:
        
        proba = model_pipeline.predict_proba(test)[:,1]
        prediction = np.where(proba > 0.4, 1, 0) # use a lower threshold
        #prediction = model_pipeline.predict(test)
        
        
        testDF1['key']=testDF1.index
        testDF1['pred']=prediction
        testDF1['prob']= proba

        indexdf['key']=indexdf.index
        merged=pd.merge(indexdf,testDF1,on='key',how='outer')  # maintain the position of the perdiction, match to the right index
        allpredict[i]=merged['pred']    # save the prediction to corresponding dataframe
        allprob[i]=merged['prob']   # save the probability
    except:
         print('something is wrong')


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle
length of data for the model: 167


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle
length of data for the model: 183


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

...

../../picklefolder_ngrams/refreshed_models/DeletionReason_not_relevant_LRApril0219.pickle
length of data for the model: 76766


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

../../picklefolder_ngrams/refreshed_models/DeletionReason_phoneORaddressORother_number_LRApril0219.pickle
length of data for the model: 76766


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

../../picklefolder_ngrams/refreshed_models/DeletionReason_preventative_screening_LRApril0219.pickle
length of data for the model: 76766


####  Part 2.b. Now get the prediction and probability dataframes and merge with testDF which has all the chart_id etc info

In [515]:
len(allpredict)

76766

In [516]:
allpredict.isnull().sum()  # a lot of nulls because not all data are run for each model

0     76599
1     76583
2     69367
3     76248
4     73545
5     75332
6     76185
7     74772
8     75576
9     70135
10    73588
11    75610
12    60476
13    75301
14    68037
15    65222
16    70621
17    76002
18    74736
19    76184
20    75238
21    71545
22        0
23        0
24        0
25        0
26        0
27        0
dtype: int64

In [517]:
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
4,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


In [518]:
allprob.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.014626,0.063301,0.160061,0.349433,0.196206,0.026679
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.005715,0.021906,0.020609,0.160427,0.018062,0.008571
2,NaN,NaN,0.113764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.008170,0.036722,0.046403,0.221087,0.024644,0.018220
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.002357,0.006155,0.004079,0.047251,0.002172,0.002319
4,NaN,NaN,0.125394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.003010,0.007203,0.021988,0.101171,0.002850,0.004171


In [519]:
len(allprob)

76766

In [520]:
len(testDF)

76766

In [521]:
# the data frame for predictions from each model
#merge to testDF to keep the Nan, which means not omputed for that instance
#testDF2=pd.concat([testDF,allpredict],axis=1)
#len(testDF2)
testDF['index']=testDF.index
allpredict['index']=allpredict.index
df_pred = pd.merge(testDF,allpredict, how='left', on = 'index')
len(df_pred)

76766

In [522]:
df_pred.tail()

,chart_id,code,text,start_offset,end_offset,text75,key,pred,prob,index,...,18,19,20,21,22,23,24,25,26,27
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...,76761,0,0.018055,76761,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...,76762,0,0.306477,76762,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...,76763,1,0.686125,76763,...,NaN,NaN,NaN,NaN,0,0,0,0,0,1
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...,76764,0,0.194581,76764,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...,76765,1,0.990072,76765,...,NaN,NaN,NaN,NaN,0,0,1,1,0,1


In [523]:
# now the data frame with probabilities
testDF['index']=testDF.index
allprob['index']=allprob.index
df_prob = pd.merge(testDF,allprob, how='left', on = 'index')
print(len(df_prob))

76766


In [524]:
df_prob.tail()

,chart_id,code,text,start_offset,end_offset,text75,key,pred,prob,index,...,18,19,20,21,22,23,24,25,26,27
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...,76761,0,0.018055,76761,...,NaN,NaN,NaN,NaN,0.272709,0.064432,0.027841,0.217822,0.013272,0.018055
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...,76762,0,0.306477,76762,...,NaN,NaN,NaN,NaN,0.012492,0.072086,0.181318,0.381113,0.030548,0.306477
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...,76763,1,0.686125,76763,...,NaN,NaN,NaN,NaN,0.018394,0.025507,0.030261,0.122995,0.003520,0.686125
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...,76764,0,0.194581,76764,...,NaN,NaN,NaN,NaN,0.011497,0.027704,0.075676,0.183114,0.005224,0.194581
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...,76765,1,0.990072,76765,...,NaN,NaN,NaN,NaN,0.154583,0.207874,0.571299,0.816210,0.020743,0.990072


#### Part 2.c. Now combine the prediction from 28 models to make one prediction,it's actually an or operation, but here using sum()

In [525]:
allprob.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,index
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.014626,0.063301,0.160061,0.349433,0.196206,0.026679,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.005715,0.021906,0.020609,0.160427,0.018062,0.008571,1
2,NaN,NaN,0.113764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.008170,0.036722,0.046403,0.221087,0.024644,0.018220,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.002357,0.006155,0.004079,0.047251,0.002172,0.002319,3
4,NaN,NaN,0.125394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.003010,0.007203,0.021988,0.101171,0.002850,0.004171,4


In [526]:
allpredictNan=allpredict   # keep the original NaN if that row is not computed for that model
allpredict=allpredict.fillna(0) # fill the NaNs by zero

In [527]:
allpredictNan.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,index
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,1
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,3
4,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,4


In [528]:
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,4


In [529]:
allpredict.drop(['index'],inplace=True, axis=1)
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [530]:
#allpredict=allpredict[[22,23,24,25,26,27]]  # only select deletion reason columns
#allpredict.columns
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [531]:
allpredict['Sum'] = allpredict.sum(axis = 1, skipna = True) 
df_pred['all_model_pred']=[1 if allpredict['Sum'].iloc[i]>=1 else 0 for i in range(len(allpredict))] 

In [532]:
# check how many 1s are predicted 
allpredict1=df_pred['all_model_pred']
print(sum(allpredict1))
print(len(allpredict1))  # 228562 are predicted as deleted out of 418310

34493
76766


In [533]:
# drop the key and predict prob temp columns
df_pred.drop(['key','pred','prob','index'],inplace=True, axis=1)
df_prob.drop(['key','pred','prob','index'],inplace=True, axis=1)

In [534]:
df_pred.tail()
#all_model_pred is check any model output is 1 for that instance

,chart_id,code,text,start_offset,end_offset,text75,0,1,2,3,...,19,20,21,22,23,24,25,26,27,all_model_pred
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,1
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,1
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,1,1
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,1,1,0,1,1


In [535]:
df_pred.iloc[135]   # just check one instance

chart_id          IBXRMA2018_MULT_1163518440010001_HMK_188344641...
code                                                            I10
text                                                            i10
start_offset                                                   1241
end_offset                                                     1244
text75            clerotic heart disease native coronary artery ...
0                                                               NaN
1                                                               NaN
2                                                               NaN
3                                                               NaN
4                                                               NaN
5                                                               NaN
6                                                               NaN
7                                                               NaN
8                                               

In [536]:
df_pred.columns

Index([      'chart_id',           'code',           'text',   'start_offset',
           'end_offset',         'text75',                0,                1,
                      2,                3,                4,                5,
                      6,                7,                8,                9,
                     10,               11,               12,               13,
                     14,               15,               16,               17,
                     18,               19,               20,               21,
                     22,               23,               24,               25,
                     26,               27, 'all_model_pred'],
      dtype='object')

In [537]:
df_prob.columns

Index([    'chart_id',         'code',         'text', 'start_offset',
         'end_offset',       'text75',              0,              1,
                    2,              3,              4,              5,
                    6,              7,              8,              9,
                   10,             11,             12,             13,
                   14,             15,             16,             17,
                   18,             19,             20,             21,
                   22,             23,             24,             25,
                   26,             27],
      dtype='object')

In [538]:
files

['../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_hypertension_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_dementia_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cholesterol_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cad_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_aneurysm_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_COPD_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Depression_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Diabetes_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Heart_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Kidney_LRApril0219.pickle',
 '../../picklefolder_ngrams/refr

In [539]:

# if running refreshed model, use this rename
# rename the columns names, has to be consistent with the above order
# since there's no 0, or 1, 17, these models didn't run because no data for them

col_dict = {0: 'Phrase_stroke', 1: 'Phrase_emphysema',2:'Phrase_hypertension',3:'Phrase_dementia',4:'Phrase_cholesterol',
           5:'Phrase_cad',6:'Phrase_aneurysm',7:'Code_COPD',8:'Code_Depression',9:'Code_Diabetes',10:'Code_Heart',
           11:'Code_Kidney',12:'Code_Model1',13:'Code_Cancer',14:'Code_Cholesterol',15:'Code_Model2',
           16:'Code_Model3',17:'Phrase_asthma',18:'Phrase_cancer',19:'Phrase_ckd',20:'Phrase_depression',21:'Phrase_diabetes',
            22:'DeletionReason_family_history',23:'DeletionReason_negation',24:'DeletionReason_not_doctors_note',
           25:'DeletionReason_not_relevant',26:'DeletionReason_phoneORaddressORother_number',27:'DeletionReason_preventative_screening',
           }   ## key→old name, value→new name

df_pred.columns = [col_dict.get(x, x) for x in df_pred.columns]
df_prob.columns = [col_dict.get(x, x) for x in df_prob.columns]


In [540]:
'''
# if running pipeline models, use this rename
# rename the columns names, has to be consistent with the above order  
# since there's no 0, or 1, 17, these models didn't run because no data for them
 ## key→old name, value→new name
col_dict = {0:'Code_Cholesterol' , 1: 'DeletionReason_family_history',2:'DeletionReason_negation',3:'DeletionReason_not_doctors_note',4:'Code_Model2',
           5:'DeletionReason_not_relevant',6:'Phrase_stroke',7:'Phrase_hypertension',8:'Phrase_emphysema',9:'Phrase_diabetes',10:'Phrase_depression',
           11:'Phrase_dementia',12:'Phrase_ckd',13:'Phrase_cholesterol',14:'Phrase_cancer',15:'Phrase_cad',
           16:'Phrase_asthma',17:'Phrase_aneurysm',18:'DeletionReason_preventative_screening',19:'DeletionReason_phoneORaddressORother_number',20:'Code_Model3',21:'Code_Model1',
            22:'Code_Kidney',23:'Code_Heart',24:'Code_Diabetes',
           25:'Code_Depression',26:'Code_COPD',27:'Code_Cancer',
           }  

df_pred.columns = [col_dict.get(x, x) for x in df_pred.columns]
df_prob.columns = [col_dict.get(x, x) for x in df_prob.columns]
'''

"\n# if running pipeline models, use this rename\n# rename the columns names, has to be consistent with the above order  \n# since there's no 0, or 1, 17, these models didn't run because no data for them\n ## key→old name, value→new name\ncol_dict = {0:'Code_Cholesterol' , 1: 'DeletionReason_family_history',2:'DeletionReason_negation',3:'DeletionReason_not_doctors_note',4:'Code_Model2',\n           5:'DeletionReason_not_relevant',6:'Phrase_stroke',7:'Phrase_hypertension',8:'Phrase_emphysema',9:'Phrase_diabetes',10:'Phrase_depression',\n           11:'Phrase_dementia',12:'Phrase_ckd',13:'Phrase_cholesterol',14:'Phrase_cancer',15:'Phrase_cad',\n           16:'Phrase_asthma',17:'Phrase_aneurysm',18:'DeletionReason_preventative_screening',19:'DeletionReason_phoneORaddressORother_number',20:'Code_Model3',21:'Code_Model1',\n            22:'Code_Kidney',23:'Code_Heart',24:'Code_Diabetes',\n           25:'Code_Depression',26:'Code_COPD',27:'Code_Cancer',\n           }  \n\ndf_pred.columns = [c

In [541]:
df_pred.columns

Index(['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75',
       'Phrase_stroke', 'Phrase_emphysema', 'Phrase_hypertension',
       'Phrase_dementia', 'Phrase_cholesterol', 'Phrase_cad',
       'Phrase_aneurysm', 'Code_COPD', 'Code_Depression', 'Code_Diabetes',
       'Code_Heart', 'Code_Kidney', 'Code_Model1', 'Code_Cancer',
       'Code_Cholesterol', 'Code_Model2', 'Code_Model3', 'Phrase_asthma',
       'Phrase_cancer', 'Phrase_ckd', 'Phrase_depression', 'Phrase_diabetes',
       'DeletionReason_family_history', 'DeletionReason_negation',
       'DeletionReason_not_doctors_note', 'DeletionReason_not_relevant',
       'DeletionReason_phoneORaddressORother_number',
       'DeletionReason_preventative_screening', 'all_model_pred'],
      dtype='object')

In [542]:
df_prob.columns

Index(['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75',
       'Phrase_stroke', 'Phrase_emphysema', 'Phrase_hypertension',
       'Phrase_dementia', 'Phrase_cholesterol', 'Phrase_cad',
       'Phrase_aneurysm', 'Code_COPD', 'Code_Depression', 'Code_Diabetes',
       'Code_Heart', 'Code_Kidney', 'Code_Model1', 'Code_Cancer',
       'Code_Cholesterol', 'Code_Model2', 'Code_Model3', 'Phrase_asthma',
       'Phrase_cancer', 'Phrase_ckd', 'Phrase_depression', 'Phrase_diabetes',
       'DeletionReason_family_history', 'DeletionReason_negation',
       'DeletionReason_not_doctors_note', 'DeletionReason_not_relevant',
       'DeletionReason_phoneORaddressORother_number',
       'DeletionReason_preventative_screening'],
      dtype='object')

In [543]:
#df_pred.rename(columns={'start': 'start_offset', 'end': 'end_offset'}, inplace=True)
#df_pred[['chart_id','start_offset','end_offset','DeletionReason_family_history']].tail()

In [544]:
#df_prob.rename(columns={'start': 'start_offset', 'end': 'end_offset'}, inplace=True)
#df_prob[['chart_id','start_offset','end_offset','DeletionReason_family_history']].tail()  #ok make sense

In [545]:
#df_pred.to_csv('run1000charts_pred_0521.csv')   #  with prediction for each model and all model combined
#df_prob.to_csv('probdataforcompwithAM_0506.csv')   #  with probability for each model
# finally corrected a lot of errors like index not reset issue

In [546]:
print(len(set(df_pred['chart_id'].tolist())))
print(len(set(df_prob['chart_id'].tolist())))   # 813 unique chart ids

980
980


###  Part 3. Load in coder feedback and  evaluate the accuracy

#### Select common chart_ids from both 1000 model run and 6000 coder annotation

In [547]:
df_pred.columns

Index(['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75',
       'Phrase_stroke', 'Phrase_emphysema', 'Phrase_hypertension',
       'Phrase_dementia', 'Phrase_cholesterol', 'Phrase_cad',
       'Phrase_aneurysm', 'Code_COPD', 'Code_Depression', 'Code_Diabetes',
       'Code_Heart', 'Code_Kidney', 'Code_Model1', 'Code_Cancer',
       'Code_Cholesterol', 'Code_Model2', 'Code_Model3', 'Phrase_asthma',
       'Phrase_cancer', 'Phrase_ckd', 'Phrase_depression', 'Phrase_diabetes',
       'DeletionReason_family_history', 'DeletionReason_negation',
       'DeletionReason_not_doctors_note', 'DeletionReason_not_relevant',
       'DeletionReason_phoneORaddressORother_number',
       'DeletionReason_preventative_screening', 'all_model_pred'],
      dtype='object')

In [556]:
# for pipeline model name list
'''
model_list=['Code_Cholesterol', 'DeletionReason_family_history',
       'DeletionReason_negation', 'DeletionReason_not_doctors_note',
       'Code_Model2', 'DeletionReason_not_relevant', 'Phrase_stroke',
       'Phrase_hypertension', 'Phrase_emphysema', 'Phrase_diabetes',
       'Phrase_depression', 'Phrase_dementia', 'Phrase_ckd',
       'Phrase_cholesterol', 'Phrase_cancer', 'Phrase_cad', 'Phrase_asthma',
       'Phrase_aneurysm', 'DeletionReason_preventative_screening',
       'DeletionReason_phoneORaddressORother_number', 'Code_Model3',
       'Code_Model1', 'Code_Kidney', 'Code_Heart', 'Code_Diabetes',
       'Code_Depression', 'Code_COPD', 'Code_Cancer']
'''
# for refreshed model 
model_list=['Phrase_stroke', 'Phrase_emphysema', 'Phrase_hypertension',
       'Phrase_dementia', 'Phrase_cholesterol', 'Phrase_cad',
       'Phrase_aneurysm', 'Code_COPD', 'Code_Depression', 'Code_Diabetes',
       'Code_Heart', 'Code_Kidney', 'Code_Model1', 'Code_Cancer',
       'Code_Cholesterol', 'Code_Model2', 'Code_Model3', 'Phrase_asthma',
       'Phrase_cancer', 'Phrase_ckd', 'Phrase_depression', 'Phrase_diabetes',
       'DeletionReason_family_history', 'DeletionReason_negation',
       'DeletionReason_not_doctors_note', 'DeletionReason_not_relevant',
       'DeletionReason_phoneORaddressORother_number',
       'DeletionReason_preventative_screening']

In [562]:

analysis_result_df=pd.DataFrame(columns=['model','TP','TN','FN','FP','precision','recall','accuracy'])

for model_name in model_list:
    df_pred['all_model_pred']=df_pred[model_name]
    print(model_name)
    TP,TN,FN,FP,precision,recall,accuracy = run_model_analysis(df_pred)
    analysis_result_df=analysis_result_df.append({'model':model_name,'TP':TP, 'TN':TN,'FN':FN,'FP':FP,'precision':precision,'recall':recall,'accuracy':accuracy},ignore_index = True)

    

Phrase_stroke


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:177: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



TP: 0
TN: 139
FN: 1
FP: 0
Phrase_emphysema
TP: 40
TN: 63
FN: 13
FP: 39
Phrase_hypertension
TP: 3556
TN: 609
FN: 27
FP: 1734
Phrase_dementia
TP: 123
TN: 150
FN: 14
FP: 91
Phrase_cholesterol
TP: 372
TN: 1671
FN: 38
FP: 387
Phrase_cad
TP: 224
TN: 614
FN: 52
FP: 177
Phrase_aneurysm
TP: 83
TN: 310
FN: 33
FP: 33
Code_COPD
TP: 896
TN: 323
FN: 67
FP: 368
Code_Depression
TP: 90
TN: 749
FN: 19
FP: 98
Code_Diabetes
TP: 1071
TN: 3033
FN: 313
FP: 806
Code_Heart
TP: 740
TN: 1173
FN: 200
FP: 394
Code_Kidney
TP: 328
TN: 207
FN: 143
FP: 196
Code_Model1
TP: 5954
TN: 3383
FN: 368
FP: 3114
Code_Cancer
TP: 48
TN: 978
FN: 32
FP: 58
Code_Cholesterol
TP: 3457
TN: 1667
FN: 89
FP: 1744
Code_Model2
TP: 4659
TN: 2131
FN: 257
FP: 2044
Code_Model3
TP: 1383
TN: 2511
FN: 149
FP: 633
Phrase_asthma
TP: 247
TN: 160
FN: 16
FP: 212
Phrase_cancer
TP: 30
TN: 1449
FN: 47
FP: 66
Phrase_ckd
TP: 183
TN: 57
FN: 7
FP: 160
Phrase_depression
TP: 143
TN: 918
FN: 46
FP: 129
Phrase_diabetes
TP: 1374
TN: 1583
FN: 162
FP: 840
DeletionRe

In [563]:
analysis_result_df

,model,TP,TN,FN,FP,precision,recall,accuracy
0,Phrase_stroke,0,139,1,0,0.922520,0.000000,0.992857
1,Phrase_emphysema,40,63,13,39,0.506329,0.754717,0.664516
2,Phrase_hypertension,3556,609,27,1734,0.672212,0.992464,0.702835
3,Phrase_dementia,123,150,14,91,0.574766,0.897810,0.722222
4,Phrase_cholesterol,372,1671,38,387,0.490119,0.907317,0.827796
5,Phrase_cad,224,614,52,177,0.558603,0.811594,0.785380
6,Phrase_aneurysm,83,310,33,33,0.715517,0.715517,0.856209
7,Code_COPD,896,323,67,368,0.708861,0.930426,0.737001
8,Code_Depression,90,749,19,98,0.478723,0.825688,0.877615
9,Code_Diabetes,1071,3033,313,806,0.570591,0.773844,0.785755


In [564]:
#analysis_result_df.to_csv('pipeline_model_1000chart_numbers0624.csv')
#analysis_result_df.to_csv('refreshed_model_1000chart_numbers0624.csv')

In [565]:
pipeline_df=pd.read_csv('pipeline_model_1000chart_numbers0624.csv')
pipeline_df

,Unnamed: 0,TP,TN,FN,FP,precision,recall,accuracy,model
0,0,3302,1959,201,1458,0.693697,0.942621,0.760260,Code_Cholesterol
1,1,24419,5559,221,29921,0.449374,0.991031,0.498636,DeletionReason_family_history
2,2,24169,11222,479,24303,0.498618,0.980566,0.588154,DeletionReason_negation
3,3,23798,13275,801,22256,0.516741,0.967438,0.616547,DeletionReason_not_doctors_note
4,4,4612,2175,294,2003,0.697203,0.940073,0.747138,Code_Model2
5,5,22377,20602,2065,15078,0.597437,0.915514,0.714863,DeletionReason_not_relevant
6,6,0,124,1,15,0.000000,0.000000,0.885714,Phrase_stroke
7,7,3469,821,93,1518,0.695609,0.973891,0.726995,Phrase_hypertension
8,8,46,72,8,32,0.589744,0.851852,0.746835,Phrase_emphysema
9,9,1296,1694,241,727,0.640633,0.843201,0.755432,Phrase_diabetes


In [566]:
analysis_result_df['model'].unique()

array(['Phrase_stroke', 'Phrase_emphysema', 'Phrase_hypertension',
       'Phrase_dementia', 'Phrase_cholesterol', 'Phrase_cad',
       'Phrase_aneurysm', 'Code_COPD', 'Code_Depression', 'Code_Diabetes',
       'Code_Heart', 'Code_Kidney', 'Code_Model1', 'Code_Cancer',
       'Code_Cholesterol', 'Code_Model2', 'Code_Model3', 'Phrase_asthma',
       'Phrase_cancer', 'Phrase_ckd', 'Phrase_depression',
       'Phrase_diabetes', 'DeletionReason_family_history',
       'DeletionReason_negation', 'DeletionReason_not_doctors_note',
       'DeletionReason_not_relevant',
       'DeletionReason_phoneORaddressORother_number',
       'DeletionReason_preventative_screening'], dtype=object)

In [570]:
# merge these two
new_df = pd.merge(pipeline_df, analysis_result_df,how = 'inner', on = ['model'], indicator = True)
new_df

,Unnamed: 0,TP_x,TN_x,FN_x,FP_x,precision_x,recall_x,accuracy_x,model,TP_y,TN_y,FN_y,FP_y,precision_y,recall_y,accuracy_y,_merge
0,0,3302,1959,201,1458,0.693697,0.942621,0.760260,Code_Cholesterol,3457,1667,89,1744,0.664680,0.974901,0.736524,both
1,1,24419,5559,221,29921,0.449374,0.991031,0.498636,DeletionReason_family_history,24638,2728,32,32729,0.429480,0.998703,0.455137,both
2,2,24169,11222,479,24303,0.498618,0.980566,0.588154,DeletionReason_negation,24550,6137,119,29329,0.455651,0.995176,0.510302,both
3,3,23798,13275,801,22256,0.516741,0.967438,0.616547,DeletionReason_not_doctors_note,24436,8152,229,27332,0.472029,0.990716,0.541788,both
4,4,4612,2175,294,2003,0.697203,0.940073,0.747138,Code_Model2,4659,2131,257,2044,0.695062,0.947722,0.746893,both
5,5,22377,20602,2065,15078,0.597437,0.915514,0.714863,DeletionReason_not_relevant,23901,15310,729,20214,0.541789,0.970402,0.651844,both
6,6,0,124,1,15,0.000000,0.000000,0.885714,Phrase_stroke,0,139,1,0,0.922520,0.000000,0.992857,both
7,7,3469,821,93,1518,0.695609,0.973891,0.726995,Phrase_hypertension,3556,609,27,1734,0.672212,0.992464,0.702835,both
8,8,46,72,8,32,0.589744,0.851852,0.746835,Phrase_emphysema,40,63,13,39,0.506329,0.754717,0.664516,both
9,9,1296,1694,241,727,0.640633,0.843201,0.755432,Phrase_diabetes,1374,1583,162,840,0.620596,0.894531,0.746906,both


In [571]:
new_df.to_csv('pipeline_vs_refreshed_model_1000charts_numbers_0624.csv')

In [552]:
# test
'''
df_pred['all_model_pred']=df_pred['Phrase_stroke']
#model_name='Code_Cholesterol'
#df_pred['all_model_pred']=df_pred['DeletionReason_family_history']
TP,TN,FN,FP,precision,recall,accuracy = run_model_analysis(df_pred)
print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)
'''

"\ndf_pred['all_model_pred']=df_pred['Phrase_stroke']\n#model_name='Code_Cholesterol'\n#df_pred['all_model_pred']=df_pred['DeletionReason_family_history']\nTP,TN,FN,FP,precision,recall,accuracy = run_model_analysis(df_pred)\nprint('TP:',TP)\nprint('TN:',TN)\nprint('FN:',FN)\nprint('FP:',FP)\nprint('precision:',precision)\nprint('recall:',recall)\nprint('accuracy:',accuracy)\n"

In [561]:
# model computation
def run_model_analysis(df_pred):
    df_pred1=df_pred[['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75','all_model_pred']]
    #print(len(df_pred1))
    df_pred1 = df_pred1.drop_duplicates()
    #print(len(df_pred1))
    # read in all charts with coder feedback
    df_codeall = pd.read_csv('/home/jovyan/work/Analytics_Data_training/conditions_with_chartids_29042019.csv') # 1000 charts
    #df_codeall = pd.read_csv('/home/jovyan/work/Analytics_Data_training/conditions_output_withchart_id_100619.csv') #7500 chart
    #print(len(df_codeall))
    #print("unique chart is in 1000 charts run model output:",df_pred1['chart_id'].nunique())
    #print("unique chart is in 6000 charts with coder feedback:",df_codeall['chart_id'].nunique())
    # now find overlap chart ids
    overlapped_charts = pd.merge(df_codeall[['chart_id']].drop_duplicates(),df_pred1[['chart_id']].drop_duplicates(), how = 'inner')
    overlapped_charts = list(overlapped_charts['chart_id'].unique())
    #print(len(overlapped_charts))  
    # select the model output
    fd_prod = df_pred1[['chart_id','code','start_offset','end_offset','text75','all_model_pred']][df_pred1['chart_id'].isin(overlapped_charts)].drop_duplicates()
    #len(fd_prod)
    # select the model output, if it's individual model, need to select those not NaN instances only
    fd_prod.dropna(inplace=True)
    #print(len(fd_prod))
    #fd_prod.isnull().sum()

    # select those in the code output
    data = df_codeall[df_codeall['chart_id'].isin(overlapped_charts)]
    #print(len(data))
    #print(len(data[data['label'] == 'added']))
    #print(len(data[data['label'] == 'deleted']))
   # print(len(data[data['label'] == 'agreed']))

    #The accurary number for the pipeline models
    # Use fuzzy logic to compute the deleted, added, agreed
    data = data[data['start'] != 0]
    data = data[data['end'] != 0]
    #data = data[data['created_at'] == date ] ##Change date to filter for datewise data
    #label_counts = data.groupby(['label']).size().reset_index(name='counts')
    instmatches = data[data['label'] == 'agreed']
    instfuzzymatches = data[data['label'] == 'deleted']
    instfuzzymatches = instfuzzymatches[instfuzzymatches['deleted_reason'].isin(['Incorrect Specification - Non-Risk Adjusted','repeated_instance','incorrect_year_of_service']) ]
    instmatches = pd.concat([instmatches, instfuzzymatches])
    instmatches = instmatches[['chart_id','submission_id','code_id','start','end']].drop_duplicates()

    instdeleted = data[data['label'] == 'deleted']
    instdeleted = instdeleted[instdeleted['deleted_reason'] != 'other']
    instdeleted = pd.merge(instdeleted, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start','end'], indicator = True)
    instdeleted = instdeleted[instdeleted['_merge'] == 'left_only']
    instdeleted = instdeleted[['chart_id','submission_id','code_id','start','end','deleted_reason']].drop_duplicates()
    deleted_reason = instdeleted.groupby(['deleted_reason']).size().reset_index(name = 'counts')
    instdeleted = instdeleted[['chart_id','submission_id','code_id','start','end']].drop_duplicates()


    instadded = data[data['label'] == 'added']
    instadded = pd.merge(instadded, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start','end'], indicator = True)
    instadded = instadded[instadded['_merge'] == 'left_only']
    instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()
    instadded = pd.merge(instadded, instdeleted,how = 'outer', on = ['chart_id','submission_id','code_id','start','end'], indicator = True)
    instadded = instadded[instadded['_merge'] == 'left_only']
    instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()

    added = len(instadded)
    deleted = len(instdeleted)
    agreed = len(instmatches)
   # print(added)
    #print(deleted)
  #  print(agreed)


  #  print(instmatches.columns)
    # my added small part
    instmatchessub=instmatches[['chart_id', 'code_id', 'submission_id','start','end']]
    instmatchessub.columns = ['chart_id', 'code_id','submission_id', 'start_offset', 'end_offset']

    instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start_offset', 'end_offset']
    prod_additions_models = pd.merge(instadded, fd_prod,how = 'inner', on = ['chart_id'], indicator = True)
    prod_additions_models['abs_diff_start'] = abs(prod_additions_models['start_offset_x'] - prod_additions_models['start_offset_y'])
    prod_additions_models['abs_diff_end'] = abs(prod_additions_models['end_offset_x'] - prod_additions_models['end_offset_y'])
    prod_additions_models1 = prod_additions_models[ (
                        (prod_additions_models['code_id'] == prod_additions_models['code']) 
                        & (prod_additions_models['abs_diff_start'] <= 75))
              | (prod_additions_models['start_offset_x'] == prod_additions_models['start_offset_y'])
              | (prod_additions_models['end_offset_x'] == prod_additions_models['end_offset_y'])
              |((prod_additions_models['code_id'] == prod_additions_models['code']) 
                        & (prod_additions_models['abs_diff_end'] <= 75))
              ]
    instadded_modeldeleted = prod_additions_models1[['chart_id','submission_id','code','start_offset_y','end_offset_y']].drop_duplicates()

  #  print(len(instadded_modeldeleted))

    # initial numbers considering model deleted additions data for recall

    total = len(instmatches) + len(instdeleted) + len(instadded_modeldeleted)
    added = len(instadded_modeldeleted)
    deleted = len(instdeleted)
    agreed = len(instmatches)

    TP = agreed
    FN = added
    FP = deleted
    TN= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))

    precision=TP/(FP+TP)
    recall=TP/(FN+TP)
    accuracy=(TP+TN)/(TP+TN1+FN+FP)

   
    # Evaluated Model accuracy numbers¶
    # select those 708 charts and only passed the filtering not deleted, 0 is keep here

    dev_df_sub_1 = fd_prod[fd_prod['all_model_pred']==0]  # select the 798 chart
    #print(len(dev_df_sub_1[['chart_id','start_offset']].drop_duplicates()))  # 37085 pass through

    dev_df_sub_1.rename(columns={'start_offset':'start','end_offset':'end'},inplace=True)
    dev_df_sub_1.rename(columns={'code':'code_id'},inplace=True)

    new_matches = pd.merge(dev_df_sub_1[['chart_id','start']], instmatches,how = 'inner', on = ['chart_id','start'], indicator = True)
    new_deleted = pd.merge(dev_df_sub_1[['chart_id','start']], instdeleted,how = 'inner', on = ['chart_id','start'], indicator = True)

    instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
    new_added = pd.merge(dev_df_sub_1, instadded,how = 'inner', on = ['chart_id'], indicator = True)
    new_added['abs_diff_start'] = abs(new_added['start_x'] - new_added['start_y'])
    new_added['abs_diff_end'] = abs(new_added['end_x'] - new_added['end_y'])

    new_added1 = new_added[ (
                        (new_added['code_id_x'] == new_added['code_id_y']) 
                        & (new_added['abs_diff_start'] <= 5))
              | (new_added['start_x'] == new_added['start_y'])
              ]

    #1. if the start offset matches exactly
    #2. If the end offset matches exactly
    #3. If the overlap of start offset is between 75 characters and the code id matches
    #4. If the overlap of end offset is between 75 characters and the code id matches
    new_added2 = new_added[ (
                        (new_added['code_id_x'] == new_added['code_id_y']) 
                        & (new_added['abs_diff_start'] <= 75))
              | (new_added['start_x'] == new_added['start_y'])
              | (new_added['end_x'] == new_added['end_y'])
              |((new_added['code_id_x'] == new_added['code_id_y']) 
                        & (new_added['abs_diff_end'] <= 75))
              ]

    #just validating the additions with a different method of calculation
    instadded_modeldeleted.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
    x = pd.merge(dev_df_sub_1[['chart_id','start']].drop_duplicates()
                 , instadded_modeldeleted[['chart_id','start']].drop_duplicates()
                 ,how = 'inner', on = ['chart_id','start'], indicator = True)

    #type 1 - older number of additions considering only start offset
    refreshed_agreements1 = int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))
    refreshed_deletions1 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))) 
    refreshed_additions1 = added - int(len(new_added1[['chart_id','start_x']].drop_duplicates()))

    #type 2 - new number of additions considering start offset and end offset
    refreshed_agreements2 = int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))
    refreshed_deletions2 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))) 
    refreshed_additions2 = added - int(len(new_added2[['chart_id','start_x']].drop_duplicates()))

    refreshed_total1 = refreshed_agreements1 + refreshed_deletions1 + refreshed_additions1
    refreshed_total2 = refreshed_agreements2 + refreshed_deletions2 + refreshed_additions2

    # my added part of calculating TN
    fd_prodsub=fd_prod[fd_prod['all_model_pred']==1]
    fd_prodsub2=fd_prod[fd_prod['all_model_pred']==0]
   # print(len(fd_prodsub))   # predicted as delete
  #  print(len(fd_prodsub2))  # not deleted, presented to coder
    fd_prodsub.columns=['chart_id', 'code_id', 'start_offset', 'end_offset', 'text75',
           'all_model_pred']   # model predicted as deleted
    fd_prodsub2.columns=['chart_id', 'code_id', 'start_offset', 'end_offset', 'text75',
           'all_model_pred']   # model predicted as passing through or agree

    instaddedsub=instadded[['chart_id', 'submission_id','code_id', 'start','end']]
    instaddedsub.columns = ['chart_id', 'submission_id', 'code_id', 'start_offset', 'end_offset']

    fd_prodsub.columns=['chart_id', 'code_id', 'start_offset', 'end_offset', 'text75',
           'all_model_pred']
    instmatchedaddedsub = pd.concat([instaddedsub,instmatchessub])
    #instmatchedaddedsub=instmatchedaddedsub[['chart_id', 'code_id', 'submission_id','start','end']]
    #instmatchedaddedsub.columns = ['chart_id', 'code_id','submission_id', 'start_offset', 'end_offset']
    #print(len(instmatchedaddedsub))  # 25273
    prod_agree_models = pd.merge(instmatchedaddedsub, fd_prodsub,how = 'inner', on = ['chart_id'], indicator = True)
   # len(prod_agree_models)

    prod_agree_models['abs_diff_start'] = abs(prod_agree_models['start_offset_x'] - prod_agree_models['start_offset_y'])
    prod_agree_models['abs_diff_end'] = abs(prod_agree_models['end_offset_x'] - prod_agree_models['end_offset_y'])

    prod_agree_models1 = prod_agree_models[ (
                        (prod_agree_models['code_id_x'] == prod_agree_models['code_id_y']) 
                        & (prod_agree_models['abs_diff_start'] <= 5))
              | (prod_agree_models['start_offset_x'] == prod_agree_models['start_offset_y'])
              | (prod_agree_models['end_offset_x'] == prod_agree_models['end_offset_y'])
              |((prod_agree_models['code_id_x'] == prod_agree_models['code_id_y']) 
                        & (prod_agree_models['abs_diff_end'] <= 5))
              ]

    instaddagreed_modeldeleted = prod_agree_models1[['chart_id','submission_id','code_id_x','code_id_y','start_offset_x','end_offset_x']].drop_duplicates()
 #   len(instaddagreed_modeldeleted)

    #type 2 calculation

 #   print('Instance level New Models:')
  #  print('Agreements = ', refreshed_agreements2*100/refreshed_total2,'%')
  #  print('Deletions = ', refreshed_deletions2*100/refreshed_total2,'%')
  #  print('Additions = ', refreshed_additions2*100/refreshed_total2,'%')
   # print('Precision = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_deletions2),'%')
    #print('Recall = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_additions2),'%')
 #   print('-------------------')
 #   print('Agreements = ', refreshed_agreements2)
 #   print('Deletions = ', refreshed_deletions2)
 #   print('Additions = ', refreshed_additions2)



    TP = refreshed_agreements2
    FP = refreshed_deletions2
    FN = refreshed_additions2

    TN=len(fd_prod[fd_prod['all_model_pred']==1])-len(instaddagreed_modeldeleted)  # deleted instances
    
    print('TP:',TP)
    print('TN:',TN)
    print('FN:',FN)
    print('FP:',FP)

    if (FP+TP)>0:
        precision=TP/(FP+TP)
    else:
        percision = 'NA'
    if (FN+TP)>0:
        recall=TP/(FN+TP)
    else:
        recall ='NA' 
        
    if (TP+TN+FN+FP)>0:
        accuracy=(TP+TN)/(TP+TN+FN+FP)
    else:
        accuracy ='NA'

    
   # print('precision:',precision)
   # print('recall:',recall)
   # print('accuracy:',accuracy)

    return TP,TN,FN,FP,precision,recall,accuracy
 

In [554]:
# save a html copy
#!jupyter nbconvert --to html all28refreshedmodels_comptoAMresult_050819-deleteinstances.ipynb